In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Kütüphanelerin Import Edilmesi 📚⬇

In [ ]:
%%time

!pip install tqdm
!pip install torch
!pip install -q -U tokenizers
!pip install -U transformers
!pip install -U simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.6 MB/s eta 0:00:00
     

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('2.0.0+cu118', '4.28.1', '0.13.3')

### Verilerin JSON dosyasından okunması 📝

In [ ]:
data_file = '/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json'

# Verilerin satır satır okunması
def get_metadata():
    with open(data_file, 'r') as f:
        for line in f:
            yield line

#### Let's print a sample paper metadata

In [ ]:
metadata = get_metadata()
for paper in metadata:
    for k, v in json.loads(paper).items():
        print(f'{k}: {v} \n')
    break

id: 0704.0001 

submitter: Pavel Nadolsky 

authors: C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan 

title: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies 

comments: 37 pages, 15 figures; published version 

journal-ref: Phys.Rev.D76:013009,2007 

doi: 10.1103/PhysRevD.76.013009 

report-no: ANL-HEP-PR-07-12 

categories: hep-ph 

license: None 

abstract:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
m

In [ ]:
kategoriler = {'astro-ph': 'Astrophysics',
                'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
                'astro-ph.EP': 'Earth and Planetary Astrophysics',
                'astro-ph.GA': 'Astrophysics of Galaxies',
                'astro-ph.HE': 'High Energy Astrophysical Phenomena',
                'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
                'astro-ph.SR': 'Solar and Stellar Astrophysics',
                'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
                'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
                'cond-mat.mtrl-sci': 'Materials Science',
                'cond-mat.other': 'Other Condensed Matter',
                'cond-mat.quant-gas': 'Quantum Gases',
                'cond-mat.soft': 'Soft Condensed Matter',
                'cond-mat.stat-mech': 'Statistical Mechanics',
                'cond-mat.str-el': 'Strongly Correlated Electrons',
                'cond-mat.supr-con': 'Superconductivity',
                'cs.AI': 'Artificial Intelligence',
                'cs.AR': 'Hardware Architecture',
                'cs.CC': 'Computational Complexity',
                'cs.CE': 'Computational Engineering, Finance, and Science',
                'cs.CG': 'Computational Geometry',
                'cs.CL': 'Computation and Language',
                'cs.CR': 'Cryptography and Security',
                'cs.CV': 'Computer Vision and Pattern Recognition',
                'cs.CY': 'Computers and Society',
                'cs.DB': 'Databases',
                'cs.DC': 'Distributed, Parallel, and Cluster Computing',
                'cs.DL': 'Digital Libraries',
                'cs.DM': 'Discrete Mathematics',
                'cs.DS': 'Data Structures and Algorithms',
                'cs.ET': 'Emerging Technologies',
                'cs.FL': 'Formal Languages and Automata Theory',
                'cs.GL': 'General Literature',
                'cs.GR': 'Graphics',
                'cs.GT': 'Computer Science and Game Theory',
                'cs.HC': 'Human-Computer Interaction',
                'cs.IR': 'Information Retrieval',
                'cs.IT': 'Information Theory',
                'cs.LG': 'Machine Learning',
                'cs.LO': 'Logic in Computer Science',
                'cs.MA': 'Multiagent Systems',
                'cs.MM': 'Multimedia',
                'cs.MS': 'Mathematical Software',
                'cs.NA': 'Numerical Analysis',
                'cs.NE': 'Neural and Evolutionary Computing',
                'cs.NI': 'Networking and Internet Architecture',
                'cs.OH': 'Other Computer Science',
                'cs.OS': 'Operating Systems',
                'cs.PF': 'Performance',
                'cs.PL': 'Programming Languages',
                'cs.RO': 'Robotics',
                'cs.SC': 'Symbolic Computation',
                'cs.SD': 'Sound',
                'cs.SE': 'Software Engineering',
                'cs.SI': 'Social and Information Networks',
                'cs.SY': 'Systems and Control',
                'econ.EM': 'Econometrics',
                'eess.AS': 'Audio and Speech Processing',
                'eess.IV': 'Image and Video Processing',
                'eess.SP': 'Signal Processing',
                'gr-qc': 'General Relativity and Quantum Cosmology',
                'hep-ex': 'High Energy Physics - Experiment',
                'hep-lat': 'High Energy Physics - Lattice',
                'hep-ph': 'High Energy Physics - Phenomenology',
                'hep-th': 'High Energy Physics - Theory',
                'math.AC': 'Commutative Algebra',
                'math.AG': 'Algebraic Geometry',
                'math.AP': 'Analysis of PDEs',
                'math.AT': 'Algebraic Topology',
                'math.CA': 'Classical Analysis and ODEs',
                'math.CO': 'Combinatorics',
                'math.CT': 'Category Theory',
                'math.CV': 'Complex Variables',
                'math.DG': 'Differential Geometry',
                'math.DS': 'Dynamical Systems',
                'math.FA': 'Functional Analysis',
                'math.GM': 'General Mathematics',
                'math.GN': 'General Topology',
                'math.GR': 'Group Theory',
                'math.GT': 'Geometric Topology',
                'math.HO': 'History and Overview',
                'math.IT': 'Information Theory',
                'math.KT': 'K-Theory and Homology',
                'math.LO': 'Logic',
                'math.MG': 'Metric Geometry',
                'math.MP': 'Mathematical Physics',
                'math.NA': 'Numerical Analysis',
                'math.NT': 'Number Theory',
                'math.OA': 'Operator Algebras',
                'math.OC': 'Optimization and Control',
                'math.PR': 'Probability',
                'math.QA': 'Quantum Algebra',
                'math.RA': 'Rings and Algebras',
                'math.RT': 'Representation Theory',
                'math.SG': 'Symplectic Geometry',
                'math.SP': 'Spectral Theory',
                'math.ST': 'Statistics Theory',
                'math-ph': 'Mathematical Physics',
                'nlin.AO': 'Adaptation and Self-Organizing Systems',
                'nlin.CD': 'Chaotic Dynamics',
                'nlin.CG': 'Cellular Automata and Lattice Gases',
                'nlin.PS': 'Pattern Formation and Solitons',
                'nlin.SI': 'Exactly Solvable and Integrable Systems',
                'nucl-ex': 'Nuclear Experiment',
                'nucl-th': 'Nuclear Theory',
                'physics.acc-ph': 'Accelerator Physics',
                'physics.ao-ph': 'Atmospheric and Oceanic Physics',
                'physics.app-ph': 'Applied Physics',
                'physics.atm-clus': 'Atomic and Molecular Clusters',
                'physics.atom-ph': 'Atomic Physics',
                'physics.bio-ph': 'Biological Physics',
                'physics.chem-ph': 'Chemical Physics',
                'physics.class-ph': 'Classical Physics',
                'physics.comp-ph': 'Computational Physics',
                'physics.data-an': 'Data Analysis, Statistics and Probability',
                'physics.ed-ph': 'Physics Education',
                'physics.flu-dyn': 'Fluid Dynamics',
                'physics.gen-ph': 'General Physics',
                'physics.geo-ph': 'Geophysics',
                'physics.hist-ph': 'History and Philosophy of Physics',
                'physics.ins-det': 'Instrumentation and Detectors',
                'physics.med-ph': 'Medical Physics',
                'physics.optics': 'Optics',
                'physics.plasm-ph': 'Plasma Physics',
                'physics.pop-ph': 'Popular Physics',
                'physics.soc-ph': 'Physics and Society',
                'physics.space-ph': 'Space Physics',
                'q-bio.BM': 'Biomolecules',
                'q-bio.CB': 'Cell Behavior',
                'q-bio.GN': 'Genomics',
                'q-bio.MN': 'Molecular Networks',
                'q-bio.NC': 'Neurons and Cognition',
                'q-bio.OT': 'Other Quantitative Biology',
                'q-bio.PE': 'Populations and Evolution',
                'q-bio.QM': 'Quantitative Methods',
                'q-bio.SC': 'Subcellular Processes',
                'q-bio.TO': 'Tissues and Organs',
                'q-fin.CP': 'Computational Finance',
                'q-fin.EC': 'Economics',
                'q-fin.GN': 'General Finance',
                'q-fin.MF': 'Mathematical Finance',
                'q-fin.PM': 'Portfolio Management',
                'q-fin.PR': 'Pricing of Securities',
                'q-fin.RM': 'Risk Management',
                'q-fin.ST': 'Statistical Finance',
                'q-fin.TR': 'Trading and Market Microstructure',
                'quant-ph': 'Quantum Physics',
                'stat.AP': 'Applications',
                'stat.CO': 'Computation',
                'stat.ME': 'Methodology',
                'stat.ML': 'Machine Learning',
                'stat.OT': 'Other Statistics',
                'stat.TH': 'Statistics Theory'}

# 1. Başlık Oluşturma
###  Özet Bilgisinden BART Modeli kullanılarak Başlık Oluşturma

#### Bu iş için, son on yılda yazılan makaleleri alalım

In [ ]:
titles = []
abstracts = []

paper_categories = np.array(list(kategoriler.keys())).flatten()

metadata = get_metadata()
for paper in tqdm(metadata):
    paper_dict = json.loads(paper)
    category = paper_dict.get('categories')
    try:
        year = int(paper_dict.get('journal-ref')[-4:])
        if category in paper_categories and 2010<year<2021:
            titles.append(paper_dict.get('title'))
            abstracts.append(paper_dict.get('abstract').replace("\n",""))
    except:
        pass 

if len(titles) != len(abstracts):
  raise Exception("Başlık ve Özet uzunlukları eşit değil")

0it [00:00, ?it/s]

### 1.2 Veri Ön İşleme ⚙️

In [ ]:
print(titles)

['A unified analysis of the reactor neutrino program towards the\n  measurement of the theta_13 mixing angle', 'The KATRIN sensitivity to the neutrino mass and to right-handed currents\n  in beta decay', 'Penguin-mediated B_(d,s)->VV decays and the Bs - anti-Bs mixing angle', 'Non Supersymmetric Metastable Vacua in N=2 SYM Softly Broken to N=1', 'Recognizing Partial Cubes in Quadratic Time', 'Chaos in Kundt type III Spacetimes', 'Strong pickup-channel coupling effects in proton scattering: the case of\n  p + Be-10', 'On the Cohomological Derivation of Yang-Mills Theory in the Antifield\n  Formalism', 'Where is the pseudoscalar glueball ?', 'Relationship between clustering and algorithmic phase transitions in the\n  random k-XORSAT model and its NP-complete extensions', 'Chiral vibrations in the A=135 region', 'Electroweak symmetry breaking and cold dark matter from strongly\n  interacting hidden sector', 'Parametric Resonance Enhancement in Neutron Interferometry and Search\n  for Non-

In [ ]:
papers = pd.DataFrame({
    'input_text': abstracts,
    'target_text': titles,
})
# Null sayısınının yazdırılması 
nan_count = papers.isna().sum() 
print("nan_count:\n",nan_count)

# Null değerlerin çıkarılması
papers = papers.dropna()

del titles, abstracts
papers

nan_count:
 input_text     0
target_text    0
dtype: int64


,input_text,target_text
0,We present in this article a detailed quanti...,A unified analysis of the reactor neutrino pro...
1,The aim of the KArlsruhe TRItium Neutrino ex...,The KATRIN sensitivity to the neutrino mass an...
2,"In this letter, we propose three different s...","Penguin-mediated B_(d,s)->VV decays and the Bs..."
3,We find non-supersymmetric metastable vacua ...,Non Supersymmetric Metastable Vacua in N=2 SYM...
4,We show how to test whether a graph with n v...,Recognizing Partial Cubes in Quadratic Time
...,...,...
19968,"Schroedinger (Nature, v.169, p.538 (1952)) d...",Real-Valued Charged Fields and Interpretation ...
19969,"Given a collection of states (rho_1, ..., rh...",How many copies are needed for state discrimin...
19970,"Hu, Paz and Zhang [ B.L. Hu, J.P. Paz and Y....",Alternative Derivation of the Hu-Paz-Zhang Mas...
19971,We demonstrate guiding of cold neutral atoms...,Guiding Neutral Atoms with a Wire


### #1.3 Model Eğitimi & Değerlendirilmesi 🚞

In [ ]:
import torch
if torch.cuda.is_available():
  print("CUDA kullanılabilir.")
else:
  print("CUDA kullanılamıyor!")

CUDA kullanılabilir.


In [ ]:
%%time

from simpletransformers.seq2seq import Seq2SeqModel

test_df = papers.sample(frac=0.1, random_state=42)
train_df = papers.drop(test_df.index)

model_args = {
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 256,
    "train_batch_size": 16,
    "num_train_epochs": 4,
}

model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

CPU times: user 6.51 s, sys: 1.98 s, total: 8.49 s
Wall time: 11.7 s


In [ ]:
from sklearn.model_selection import KFold

n_splits = 5

# Argümanlar
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 256,
    "train_batch_size": 16,
    "num_train_epochs": 12,
}

# Model oluşturma
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

kf = KFold(n_splits=n_splits)

results = []

for train_index, val_index in kf.split(papers):
    train_df = papers.iloc[train_index]
    val_df = papers.iloc[val_index]

    model.train_model(train_df, eval_data=val_df)

    result = model.eval_model(val_df)

    print(result)

    results.append(result)

In [ ]:
%%time

model.train_model(train_df)

  0%|          | 0/17976 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/4494 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/4494 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/4494 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/4494 [00:00<?, ?it/s]

CPU times: user 38min 18s, sys: 35.3 s, total: 38min 54s
Wall time: 41min 11s


(17976, 0.8800155382875302)

In [ ]:
result = model.eval_model(test_df)
print(result)

  0%|          | 0/1997 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.8199913252592087}


### Çapraz Doğrulama (Cross Validation)

In [ ]:
from sklearn.model_selection import KFold

# Define the number of folds
n_splits = 3

# Define the model arguments
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "max_seq_length": 256,
    "train_batch_size": 16,
    "num_train_epochs": 1,
}

# Create a Bart-base model
model = Seq2SeqModel(encoder_decoder_type="bart",
                    encoder_decoder_name="facebook/bart-base",
                    args=model_args)

# Define the KFold object
kf = KFold(n_splits=n_splits)

# Define a list to store the results for each iteration
results = []

# Loop over the folds
for train_index, val_index in kf.split(papers):
    # Get the training and validation data for this fold
    train_df = papers.iloc[train_index]
    val_df = papers.iloc[val_index]

    # Train the model on this fold
    model.train_model(train_df, eval_data=val_df)

    # Evaluate the model on this fold
    result = model.eval_model(val_df)

    # Print the result for this fold
    print(result)

    # Add the result to the list of results
    results.append(result)


In [ ]:
average_cv_score = sum(result['eval_loss']) / len(result['eval_loss'])
print(average_cv_score)

### #1.4 Tahmin 🔮

In [ ]:
%%capture
predicted_titles = []
true_titles = []

for index in range(len(test_df)-1):

    abstract = test_df.iloc[index]['input_text']
    true_title = test_df.iloc[index]['target_text']
    true_titles.append(true_title)

    predicted_title = model.predict([abstract])[0]
    predicted_titles.append(predicted_title)



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=37d51369f45a714dfed5c4c5422dca9502bbb3bc59de9c299dc793a186120f69
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
import evaluate

#use_aggregator : if False;Her prediction için ayrı ayrı skorlar verilir. if True;Ortalaması alınır.
#use_stemmer : if True;Kelimenin köklerine indirgeyerek, kök halleri kullanılarak aynı olarak kabul edileceği anlamına gelir.
#if False; Metindeki her kelimeyi tam olarak dikkate alır.

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=predicted_titles,
                        references=true_titles,
                        use_aggregator=True,
                        use_stemmer=False
                       )
print(results)

#ROUGE1, yalnızca özet ve referans özeti arasındaki tam eşleşen 1-gram kelimeleri kullanırken
#ROUGE2, 2-gram kelimeleri kullanır.
#ROUGEL, özet ve referans özeti arasındaki en uzun ortak alt dizgiyi bulmak için Longest Common Subsequence (LCS) kullanırken
#ROUGE Lsum, ROUGE L için farklı L uzunlukları için sonuçları birleştirir.

#ROUGE1, ROUGE2 ve ROUGE L ölçüleri, özetlemenin farklı yönlerini değerlendirir
#Bir özetlemenin kalitesini ölçmek için birbirleriyle birlikte kullanılır
#ROUGE Lsum, farklı L uzunlukları için sonuçları birleştirerek, özetlemenin genel kalitesini ölçmek için kullanılabilir.

{'rouge1': 0.3413028154292642, 'rouge2': 0.1662079043086691, 'rougeL': 0.3081338445523377, 'rougeLsum': 0.3145655499526561}


In [ ]:
import evaluate

#max_order : Buradaki girilecek olan sayı kaçsa n-gram'larına bakarak bir başarı üretir.
#smooth : Yüksek smooth değerleri, uzun çevirilerin performansını artırırken kısa çevirilerin performansını düşürebilir.
#Düşük smooth değerleri ise kısa çevirilerin performansını artırırken uzun çevirilerin performansını düşürebilir.
#0 ya da 1 tercih edilir.
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predicted_titles,
                       references=true_titles,
                       max_order=4,
                       smooth=False
                      )
print(results)

#bleu : BLEU puanı, 0 ile 1 arasında bir değerdir.
#precisions : "precisions" değeri, farklı n-gram düzeylerindeki precision değerlerini içeren bir dizi veya sözlüktür.
#brevity_penalty: Brevity penalty, çeviri cümlesinin referans cümleye göre ne kadar kısa olduğunu dikkate alır
#Bu, çeviri cümlesi uzunluğunun referans cümlesine göre ne kadar farklı olduğunu ölçer ve bir ceza uygular.
#length_ratio : Çeviri cümlesinin uzunluğu ile referans cümlesinin uzunluğu arasındaki oranı gösterir. Daha kısa bir çeviri cümlesi için bu oran daha azdır.
#translation_length : Çeviri cümlesinin uzunluğunu temsil eder. Bu, çeviri cümlesindeki toplam kelime veya karakter sayısını ifade edebilir.
#reference_length : Referans cümlesinin uzunluğunu temsil eder. Bu, referans cümlesindeki toplam kelime veya karakter sayısını ifade edebilir.

{'bleu': 0.077807248959546, 'precisions': [0.29635906726667055, 0.1290770757525637, 0.07317630916990624, 0.046202247191011236], 'brevity_penalty': 0.7296168006944208, 'length_ratio': 0.7603199289046878, 'translation_length': 17111, 'reference_length': 22505}
